In [1]:
import pandas as pd
import numpy as np

filename = '/Users/libao/Documents/data/twitter/csv/overlap_matrix_libaoutrage_macOS_parallel_block_2019_03_28_09_50_44_adjacency.csv'
df = pd.read_csv(filename,index_col=0)
df

,379005817,162569725,1007122009,19268550,16543984,741621378,260497847,1335419636,464331848,963586249,...,8069562,322818072,2308501678,2157926932,1058482008,1423312248,27380147,2495098880,1250625278,546942813
id,,,,,,,,,,,,,,,,,,,,,
379005817,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
162569725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1007122009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19268550,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
16543984,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
741621378,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260497847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1335419636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
464331848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def bfs_array(raw_graph):
    '''
    Breadth First Search on array
    Find components of a graph
    '''
    visited = [False for i in range(len(raw_graph))]
    components = []
    for i in range(len(raw_graph)):
        if not visited[i]:
            visited[i] = True
            component = [i]
            queue = []
            for j in range(len(raw_graph[i])):
                if not visited[j] and raw_graph[i, j] > 0:
                    visited[j] = True
                    queue.append(j)
                    component.append(j)
            while len(queue) > 0:
                j = queue.pop(0)
                for k in range(len(raw_graph[j])):
                    if not visited[k] and raw_graph[j, k] > 0:
                        visited[k] = True
                        component.append(k)
                        queue.append(k)
            components.append(component)
    return components

In [5]:
def bfs_df(raw_graph):
    '''
    Breadth First Search on pandas.DataFrame
    Find components of a graph
    '''
    n, _ = raw_graph.shape
    visited = [False for i in range(n)]
    components = []
    for i in range(n):
        if not visited[i]:
            visited[i] = True
            component = [i]
            queue = []
            for j in range(n):
                if not visited[j] and raw_graph.iloc[i, j] > 0:
                    visited[j] = True
                    queue.append(j)
                    component.append(j)
            while len(queue) > 0:
                j = queue.pop(0)
                for k in range(n):
                    if not visited[k] and raw_graph.iloc[j, k] > 0:
                        visited[k] = True
                        component.append(k)
                        queue.append(k)
            components.append(component)
    return components

In [6]:
def APD(A):
    '''
    Seidel's algorithm
    Find distance matrix of a connected graph
    '''
    n, _ = A.shape
    B = A
    while not all(B[i,j] for i in range(n) for j in range(n) if i != j):
        Z = B.dot(B)
        B = np.array([[1 if i != j and ( B[i,j] == 1 or Z[i,j] > 0 ) else 0 for j in range(n)] for i in range(n)])
    T = B
    X = T.dot(A)
    degree = [sum( A[i,j] for j in range(n) ) for i in range(n)]
    D = np.array([[2 * T[i,j] if X[i,j] >= T[i,j] * degree[j] else 2 * T[i,j] - 1 for j in range(n)] for i in range(n) ] )
    return D

In [7]:
def assemble_index_columns(list_name):
    '''
    Reorder index/columns of pandas.DataFrame
    '''
    idx = list_name[0].index
    col = list_name[0].columns
    idx_list = [list_name[0].index]
    col_list = [list_name[0].columns]
    for i in range(1, len(list_name)):
        idx = idx.append(list_name[i].index)
        col = col.append(list_name[i].columns)
        idx_list.append(list_name[i].index)
        col_list.append(list_name[i].columns)

    return (idx, col, idx_list, col_list)

In [8]:
# Data cleaning
col = [idx for idx in df.iloc[0].index if df.iloc[0][idx] == -1]
idx = [int(idx) for idx in col]
df = df.drop(index=idx, columns=col)
df

components = bfs_df(df)
df_dist_list = []
index_list = []
columns_list = []
for i in range(len(components)):
    idx = df.index[components[i]]
    col = df.columns[components[i]]
    D = APD(np.array(df.loc[idx,col]))
    df_dist_list.append(pd.DataFrame(data=D.astype(int), columns=col, index=idx))

df_dist_list.sort(key=len, reverse=True)
idx, col, idx_list, col_list = assemble_index_columns(df_dist_list)

df_orig = np.zeros(df.shape)
df_orig = pd.DataFrame(data=df_orig.astype(int), columns=col, index=idx)
df_dist = np.ones(df.shape) * -1
df_dist = pd.DataFrame(data=df_dist.astype(int), columns=col, index=idx)
for i in range(len(idx_list)):
    df_dist.loc[idx_list[i], col_list[i]] = df_dist_list[i]
    df_orig.loc[idx_list[i], col_list[i]] = df.loc[idx_list[i], col_list[i]]